# Generate data based on BrainWeb images

From brainweb, get: 
- Two PET images
    - FDG
    - Amyloid
- Two MR acquisitions:
    - T1
    - T2
- A $\mu$-map

We're going to do various things with the images to create some data we can play around with! In image space, this includes:
- adding misalignment to some images (amyloid and its $\mu$-map)
- adding tumours

And then forward projecting all of this data to end up with:
- Noisy and noiseless sinograms with and without misalignment, and with and without a tumour

This data is used in some of the other synergistic notebooks.

Acquiring the brainweb data is done via Casper da Costa-Luis' wrapper as before.

Authors: Richard Brown, Casper da Costa-Luis, Kris Thielemans  
First version: 2nd of November 2019  
Seconf version: June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF)  
Copyright 2019, 2021  University College London  
Copyright 2019  King's College London  

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging.
(http://www.synerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

In [ ]:
%matplotlib widget

# Setup the working directory for the notebook
import notebook_setup

import brainweb
from brainweb import volshow
import numpy as np
from os import path, mkdir, chdir, makedirs
from tqdm.auto import tqdm
import logging
logging.basicConfig(level=logging.INFO)
import nibabel as nib
import sirf.STIR as pet
import matplotlib.pyplot as plt
import sirf.Reg as reg
from math import cos, sin, pi
from sirf.Utilities import examples_data_path
from sirf_exercises import exercises_data_path
import shutil
from scipy.ndimage import gaussian_filter

wd = exercises_data_path('working_folder', 'Synergistic', 'BrainWeb')
makedirs(wd, exist_ok=True)
chdir(wd)

In [ ]:
msg_red = pet.MessageRedirector('info.txt', 'warnings.txt')

## Get brainweb data (just single patient)

In [ ]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1)
    vol_amyl = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1,
        PetClass=brainweb.Amyloid)

FDG_arr  = vol['PET']
amyl_arr = vol_amyl['PET']
uMap_arr = vol['uMap']
T1_arr   = vol['T1']
T2_arr   = vol['T2']

## Display it

In [ ]:
def subplot_(idx,vol,title,clims=None,cmap="viridis"):
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")

plt.figure();
slice_show = FDG_arr.shape[0]//2
subplot_([2,3,1],FDG_arr [slice_show, 100:-100, 100:-100],'FDG'    ,cmap="hot")
subplot_([2,3,2],amyl_arr[slice_show, 100:-100, 100:-100],'Amyloid',cmap="hot")
subplot_([2,3,3],uMap_arr[slice_show, 100:-100, 100:-100],'uMap'   ,cmap="bone")
subplot_([2,3,4],T1_arr  [slice_show, 100:-100, 100:-100],'T1'     ,cmap="Greys_r")
subplot_([2,3,5],T2_arr  [slice_show, 100:-100, 100:-100],'T2'     ,cmap="Greys_r")

## Crop images and save

Here's what's going on in this cell:

1. The data from brainweb is (127,344,344), but we want it to be (127,285,285). So just keep the middle sections of the image in the x-y plane.
2. Save the image to file.
3. Crop the image yet again to reduce it to (127,150,150). You can use either of these two sets of images, but it'll be faster to use the smaller image. We'll also apply a shift of (25,25) in the x-y plane to re-centre the image.
4. Save the smaller image to file, too.

N.B.: This requires you to have a version of SIRF > v2.1.0. See the cell at the bottom of this notebook if you have an older version of SIRF.

In [ ]:
# We'll need a template sinogram
mMR_template_sino = examples_data_path('PET') + "/mMR/mMR_template_span11.hs"
templ_sino = pet.AcquisitionData(mMR_template_sino)

def crop_and_save(templ_sino, vol, fname):
    # Crop from (127,344,344) to (127,285,285) and save to file
    vol = vol[:,17:17+285,17:17+285]
    im = pet.ImageData(templ_sino)
    im.fill(vol)
    im.write(fname)
    # Create an optional smaller version, (127,150,150)
    # For extra speeeed.
    # Also shift by (25,25) in (x,y) to recentre the image
    im = im.zoom_image(size=(-1,150,150),offsets_in_mm=(0,25,25))
    im = im.move_to_scanner_centre(templ_sino)
    im.write(fname + "_small.hv")
    return im
    
FDG  = crop_and_save(templ_sino, FDG_arr,  "FDG"    )
amyl = crop_and_save(templ_sino, amyl_arr, "Amyloid")
uMap = crop_and_save(templ_sino, uMap_arr, "uMap"   )
T1   = crop_and_save(templ_sino, T1_arr,   "T1"     )
T2   = crop_and_save(templ_sino, T2_arr,   "T2"     )

In [ ]:
plt.figure();
slice_show = FDG.as_array().shape[0]//2
subplot_([2,3,1],FDG.as_array() [slice_show,:,:],'FDG'    ,cmap="hot")
subplot_([2,3,2],amyl.as_array()[slice_show,:,:],'Amyloid',cmap="hot")
subplot_([2,3,3],uMap.as_array()[slice_show,:,:],'uMap'   ,cmap="bone")
subplot_([2,3,4],T1.as_array()  [slice_show,:,:],'T1'     ,cmap="Greys_r")
subplot_([2,3,5],T2.as_array()  [slice_show,:,:],'T2'     ,cmap="Greys_r")

## Forward project

Forward project both the FDG and amyloid images both with and without Poisson noise.

In [ ]:
def get_acquisition_model(templ_sino, uMap, global_factor=.01):
    '''create an acq_model given a mu-map and a global sensitivity factor
    
    The default global_factor is chosen such that the mean values of the
    forward projected BrainWeb data have a reasonable magnitude
    '''
    #%% create acquisition model
    am = pet.AcquisitionModelUsingRayTracingMatrix()
    # Let's use a fairly large number of rays to have a more realistic model
    am.set_num_tangential_LORs(10)

    # Set up sensitivity due to attenuation
    asm_attn = pet.AcquisitionSensitivityModel(uMap, am)
    asm_attn.set_up(templ_sino)
    bin_eff = templ_sino.get_uniform_copy(global_factor)
    print('applying attenuation (please wait, may take a while)...')
    asm_attn.unnormalise(bin_eff)
    asm = pet.AcquisitionSensitivityModel(bin_eff)

    am.set_acquisition_sensitivity(asm)
    return am

In [ ]:
# Function for adding noise
def add_noise(proj_data,noise_factor = 1):
    proj_data_arr = proj_data.as_array() / noise_factor
    # Data should be >=0 anyway, but add abs just to be safe
    proj_data_arr = np.abs(proj_data_arr)
    noisy_proj_data_arr = np.random.poisson(proj_data_arr).astype('float32');
    noisy_proj_data = proj_data.get_uniform_copy()
    noisy_proj_data.fill(noisy_proj_data_arr);
    return noisy_proj_data

In [ ]:
am = get_acquisition_model(templ_sino, uMap)
am.set_up(templ_sino, FDG)

# FDG
sino_FDG = am.forward(FDG)
sino_FDG.write("FDG_sino")
sino_FDG_noisy = add_noise(sino_FDG)
sino_FDG_noisy.write("FDG_sino_noisy")

# Amyloid
sino_amyl = am.forward(amyl)
sino_amyl.write("amyl_sino")
sino_amyl_noisy = add_noise(sino_amyl)
sino_amyl_noisy.write("amyl_sino_noisy")

In [ ]:
plt.figure();
subplot_([2,2,1],       sino_FDG.as_array()[0,60,:,:],'FDG'          )
subplot_([2,2,2], sino_FDG_noisy.as_array()[0,60,:,:],'Noisy FDG'    )
subplot_([2,2,3],      sino_amyl.as_array()[0,60,:,:],'Amyloid'      )
subplot_([2,2,4],sino_amyl_noisy.as_array()[0,60,:,:],'Noisy amyloid')

# Add a rigid transformation to the amyloid image.
Here we illustrate how to reposition the images using `sirf.Reg`. This data could be used to check what happens if there is a misalignment with the anatomical image in guided reconstruction, or between 2 PET images (see the [Dual_PET notebook](Dual_PET.ipynb). You could just come back to this notebook when you need this data of course.

As with the crop, moving an image around affects its offset in STIR, which currently causes problems. So again, we recentre the image after the resample.

In [ ]:
def add_misalignment(transformation_matrix,image):

    # Resample
    resampler = reg.NiftyResample()
    resampler.set_interpolation_type_to_cubic_spline()
    resampler.set_reference_image(image)
    resampler.set_floating_image(image)
    resampler.set_padding_value(0)
    resampler.add_transformation(transformation_matrix)
    resampler.process()

    # Save to file
    resampled = resampler.get_output()

    # Remove all offset info (avoids problems in STIR)
    misaligned_image = resampled.move_to_scanner_centre(templ_sino)
    return misaligned_image

### Create the transformation matrix

The rotation matrix we'll use here is a rotation of 30 degrees about one of the axes, and a translation of 20 and -10 mm in the x- and y-directions, respectively.

In [ ]:
# Rotation matrix
r = 10*pi/180
t_x = 20
t_y = -10

tm = reg.AffineTransformation(np.array(\
        [[ cos(r), sin(r), 0, t_x], \
         [-sin(r), cos(r), 0, t_y], \
         [      0,      0, 1, 0  ], \
         [      0,      0, 0, 1  ]]))

amyl_misaligned = add_misalignment(tm,amyl)
uMap_misaligned = add_misalignment(tm,uMap)

amyl_misaligned.write("amyl_misaligned")
uMap_misaligned.write("uMap_misaligned")

plt.figure()
subplot_([2,2,1],amyl.as_array()[60,:,:],'Amyloid')
subplot_([2,2,2],uMap.as_array()[60,:,:],'uMap')
subplot_([2,2,3],amyl_misaligned.as_array()[60,:,:],'Resampled Amyloid')
subplot_([2,2,4],uMap_misaligned.as_array()[60,:,:],'Resampled uMap')


Forward project this data. Note that we need a new acquisition model, as the attenuation image has changed.

In [ ]:
# Get acquisition model for resampled data
am_misaligned = get_acquisition_model(templ_sino, uMap_misaligned)
am_misaligned.set_up(templ_sino, amyl_misaligned)
# Forward project again
sino_amyl_misaligned = am_misaligned.forward(amyl_misaligned)
sino_amyl_misaligned.write("amyl_sino_misaligned")
sino_amyl_noisy_misaligned = add_noise(sino_amyl_misaligned)
sino_amyl_noisy_misaligned.write("amyl_sino_noisy_misaligned")

In [ ]:
plt.figure()
subplot_([2,2,1],sino_amyl.as_array()[0,60,:,:],'Amyloid')
subplot_([2,2,2],sino_amyl_noisy.as_array()[0,60,:,:],'Noisy amyloid')
subplot_([2,2,3],sino_amyl_misaligned.as_array()[0,60,:,:],'Amyloid resampled')
subplot_([2,2,4],sino_amyl_noisy_misaligned.as_array()[0,60,:,:],'Noisy resampled amyloid')

# Insert tumour

We add a spherical tumour into the FDG image. Then we forward project it and add Poisson noise. The results of would be useful to check the effect in synergistic (or guided) reconstruction. The data is currently used in the HKEM notebook.

In [ ]:
# Start with an image filled with zeroes. 
tumour_arr = FDG.get_uniform_copy(0).as_array()
# The value of the tumour will be 1.2*the max in the FDG image
tumour_val = 1.2 * FDG.max()
# Give the radius of the tumour
tumour_radius_in_voxels = 4
# Amount of smoothing
gaussian_sigma = 1
# Index of centre of the tumour
tumour_centre = np.array([60, 50, 90])
# Loop over all voxels in the cube containing the sphere
for i in range(-tumour_radius_in_voxels, tumour_radius_in_voxels):
    for j in range(-tumour_radius_in_voxels, tumour_radius_in_voxels):
        for k in range(-tumour_radius_in_voxels, tumour_radius_in_voxels):
            # If the index is inside of the sphere, set the tumour value
            if (i*i+j*j+k*k < tumour_radius_in_voxels*tumour_radius_in_voxels):
                tumour_arr[tumour_centre[0]+i,tumour_centre[1]+j,tumour_centre[2]+k] = tumour_val

# Smooth the tumour image
tumour_arr = gaussian_filter(tumour_arr, sigma=gaussian_sigma)

# Overwrite add
tumour_arr = np.max([FDG.as_array(),tumour_arr],axis=0)

# Fill into new ImageData object
pet_tumour = FDG.clone()
pet_tumour.fill(tumour_arr)
pet_tumour.write('FDG_tumour')

# Show side by side
plt.figure();
subplot_([1,2,1],FDG.as_array()[60,:,:],"PET without tumour", [0,tumour_arr.max()])
subplot_([1,2,2],tumour_arr[60,:,:],"PET tumour",[0,tumour_arr.max()])

In [ ]:
#Forward project FDG image with tumour
umap_small=pet.ImageData('uMap_small.hv')
am = get_acquisition_model(templ_sino, umap_small)

In [ ]:
# FDG
am.set_up(templ_sino,pet_tumour)
sino_tumour_FDG = am.forward(pet_tumour)
sino_tumour_FDG.write("FDG_tumour_sino")
sino_tumour_FDG_noisy = add_noise(sino_tumour_FDG)
sino_tumour_FDG_noisy.write("FDG_tumour_sino_noisy")

## Cropping with SIRF <= v2.1.0

`zoom_image` and `move_to_scanner_centre` didn't exist prior to SIRF v2.1.0. If your version is older see this link for some help: https://github.com/SyneRBI/SIRF-Exercises/issues/52. Good luck, soldier!